[![Binder](img/badge-binder.svg)](https://mybinder.org/v2/gh/nhirschey/teaching/gh-pages?filepath=assignment-risky-weights.ipynb)&emsp;
[![Script](img/badge-script.svg)](/Teaching//assignment-risky-weights.fsx)&emsp;
[![Notebook](img/badge-notebook.svg)](/Teaching//assignment-risky-weights.ipynb)

Group Name:

Student Name | Student Number
--- | ---
**1:Bernardo Manarte** |55810 &#32;
**2:Leonor Rodrigues Pereira** |48778 &#32;
**3:Marouan Kamoun** |53833 &#32;
**4:Rodrigo Simões** |53154 &#32;
**5:Vasco Calxito** |53960 &#32;


This is an assignment. You may work in groups. Please write your group and group member names above. You will find sections labeled **Task** asking you to do each piece of analysis. Please make sure that you complete all of these tasks. I included some tests to help you see if you are calculating the solution correctly, but if you cannot get the test to pass submit your best attempt and you may recieve partial credit.

All work that you submit should be your own. Make use of the course resources and example code on the course website. It should be possible to complete all the requested tasks using information given below or somewhere on the course website.



In [2]:
#r "nuget:FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: Quotes.YahooFinance"
#r "nuget: Plotly.NET, 3.*"


Installed Packages FSharp.Data, 5.0.2 FSharp.Stats, 0.4.11 Plotly.NET, 3.0.1 Quotes.YahooFinance, 0.0.5

In [3]:
#r "nuget: Plotly.NET.Interactive, 3.*"


Installed Packages Plotly.NET.Interactive, 3.0.2

Loading extensions from `/Users/rodrigosimoes/.nuget/packages/plotly.net.interactive/3.0.2/interactive-extensions/dotnet/Plotly.NET.Interactive.dll`

In [4]:
open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats
open Quotes.YahooFinance


for testing.



In [5]:
#r "nuget: FsUnit.Xunit"
#r "nuget: xunit, 2.*"
open Xunit
open FsUnit.Xunit
open FsUnitTyped


Installed Packages FsUnit.Xunit, 5.2.0 xunit, 2.4.2

## Load Data

We get the SPY ETF



In [6]:
type MonthlyReturn = { Date: DateTime; Return: float }
let spy = 
    YahooFinance.History("SPY", 
                         startDate = DateTime(2010,1,1), 
                         endDate = DateTime(2023, 2, 28),
                         interval = Monthly)
    |> List.sortBy (fun month -> month.Date)
    |> List.pairwise
    |> List.map (fun (m0, m1) -> 
        { Date = m1.Date
          Return = (m1.AdjustedClose - m0.AdjustedClose) / m0.AdjustedClose })


Load risk-free rate



In [7]:
// 4-week Treasury Bill: Secondary Market Rate
type DTB4WK = CsvProvider<"https://fred.stlouisfed.org/graph/fredgraph.csv?id=DTB4WK",
                           Schema="Date,RiskFreeRate (float)",
                           MissingValues=".">


// We'll take the 4-week interest rate at the start of the month as the risk-free rate for that month.
// Then we'll put it in a dictionary for efficient lookup.
let rf =
    DTB4WK.GetSample().Rows
    |> Seq.toList
    |> List.filter (fun x -> not (Double.IsNaN x.RiskFreeRate))
    |> List.groupBy (fun day -> day.DATE.Year, day.DATE.Month)
    |> List.map (fun (month, daysInMonth) ->
        let firstDay = 
            daysInMonth 
            |> List.sortBy (fun day -> day.DATE)
            |> List.head
        let date = DateTime(firstDay.DATE.Year, firstDay.DATE.Month, 1)
        // discount basis assumes 30 days in a month, 360 days per year.
        let ret = (30.0 / 360.0) * firstDay.RiskFreeRate / 100.0 
        date, ret)
    |> dict

Look at an example, the index is date



In [8]:
rf[DateTime(2010,1,1)] 


4.1666666666666665E-05

Calculating excess returns of spy returns:



In [9]:
let excessSpy =
    [ for x in spy do 
        { Date = x.Date
          Return = x.Return - rf.[x.Date] } ]
excessSpy

index,Date,Return
0,2010-02-01 00:00:00Z,0.03115321017649878
1,2010-03-01 00:00:00Z,0.056462082617906924
2,2010-04-01 00:00:00Z,0.01951824745444261
3,2010-05-01 00:00:00Z,-0.07957124309331076
4,2010-06-01 00:00:00Z,-0.05634761708539586
5,2010-07-01 00:00:00Z,0.07324966983499075
6,2010-08-01 00:00:00Z,-0.04509684796264613
7,2010-09-01 00:00:00Z,0.08361961684966472
8,2010-10-01 00:00:00Z,0.04363499399638644
9,2010-11-01 00:00:00Z,-0.00011666666666666667


As an example, I'll first calculate the standard deviation of the the excess returns of SPY and assign it to a value named `stdDevExcessSpy`.



In [10]:
let stdDevExcessSpy =
    excessSpy
    |> stDevBy (fun x -> x.Return)
stdDevExcessSpy

0.04384990509920313

The following test will pass if I calculate it correctly.



In [11]:
// Test.
if true then
    stdDevExcessSpy
    |> should (equalWithin 0.005) 0.04

The test following test will fail if I calculate it incorrectly. In this failing example I report an annualized standard deviation instead of a monthly standard deviation.



In [12]:
let stdDevFAIL =
    let monthlyStDev = 
        excessSpy
        |> stDevBy (fun x -> x.Return)
    monthlyStDev * (sqrt 12.0)

// Test
if true then // make this `if true` to run the test.
    stdDevFAIL
    |> should (equalWithin 0.005) 0.04



Error: FsUnit.Xunit+MatchException: Exception of type 'FsUnit.Xunit+MatchException' was thrown.
Expected: 0.04 with a tolerance of 0.005
Actual:   0.1519005271
   at FsUnit.Xunit.Assert.That.Static[a](a actual, IMatcher`1 matcher) in C:\Users\const\github\repos\FsUnit\src\FsUnit.Xunit\FsUnit.fs:line 30
   at <StartupCode$FSI_0016>.$FSI_0016.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

## Start of the assignment

> **Task:** What is the cumulative return of SPY from the beginning to the end of the sample period? In other words, if you invest
$1 in SPY at the beginning of the sample, how many **additional** dollars would you have by the end? Assign it to a value named `cumulativeSpyReturn`.
> 

Write your solution in the cell below.



In [13]:
// Solution here
let cumulativeReturn =
    let cumulativeSpyReturn =
        let mutable cumprod = 1.0
        let returns = spy |> List.map (fun row -> row.Return)
        for r in returns do
            cumprod <- cumprod * (1.0 + r)
        cumprod - 1.0
    printfn $"The cumulative return of SPY from the beginning to the end of the sample period is {cumulativeSpyReturn}."
    cumulativeSpyReturn
        
// Test
if true then
    cumulativeReturn 
    |> should (equalWithin 0.1) 3.75
    printfn "Test passed"

The cumulative return of SPY from the beginning to the end of the sample period is 3,727623349121326.
Test passed


> **Task:** What is the cumulative **excess** return of SPY from the beginning to the end of the sample period? Assign it to a value named `cumulativeExcessSpyReturn`.
> 

Write your solution in the cell below.



In [14]:
// Solution here
let cumulativeExcessReturn =
    let cumulativeExcessSpyReturn =
        let mutable cumprod = 1.0
        let returns = excessSpy |> List.map (fun row -> row.Return)
        for r in returns do
            cumprod <- cumprod * (1.0 + r)
        cumprod - 1.0
    printfn $"The excess cumulative return of SPY from the beginning to the end of the sample period is {cumulativeExcessSpyReturn}."
    cumulativeExcessSpyReturn
// Test

if true then
    cumulativeExcessReturn 
    |> should (equalWithin 0.1) 3.3
    printfn "Test passed"


The excess cumulative return of SPY from the beginning to the end of the sample period is 3,369733908516112.
Test passed


> **Task:** Plot the cumulative **excess** return of SPY from the beginning to the end of the sample period. The date should be the x-axis and the cumulative excess return should be the y-axis.
> 

```
No value returned by any evaluator
```

Write your solution in the cell below.



In [15]:
// Solution here
let excessrplot = 
    let mutable cumprod = 1.0
    [ for row in excessSpy do
        cumprod <- cumprod * (1.0 + row.Return) 
        let dataPoint = (row.Date, cumprod-1.0)
        yield dataPoint ]

Chart.Line(excessrplot)

<!-- Plotly chart will be drawn inside this DIV -->

> **Task:** Calculate the standard deviation of the `excessSPY` monthly returns from 2010-01 to 2019-12 (inclusive). Assign it to a value named `stdDev2010s`.
> 

Write your solution in the cell below.



In [16]:
// Solution here
let excessSpyMonthly =
    excessSpy
    |> Seq.filter (fun x -> x.Date >= DateTime(2010, 1, 1) && x.Date <= DateTime(2019, 12, 31))
    |> Seq.groupBy (fun x -> x.Date.Year, x.Date.Month)
    |> Seq.map (fun (_, group) -> group |> Seq.map (fun x -> x.Return) |> Seq.average)
    |> Seq.toList

let stdDev2010s =
    excessSpyMonthly
    |> stDev

// Test
stdDev2010s 
|> should (equalWithin 0.005) 0.035

stdDev2010s

0.03657056671199984

> **Task:** Calculate the average monthly excess returns of SPY from 2010-01 to 2019-12 (inclusive). Assign it to a value named `mu2010s`.
> 

Write your solution in the cell below.



In [17]:
// Solution here
let excessSpyMonthlyAGV =
    [ for x in spy do 
        { Date = x.Date
          Return = x.Return - rf.[x.Date] } ]
    |> List.filter (fun x -> x.Date >= DateTime(2010,1,1) && x.Date <= DateTime(2019,12,31))

let mu2010s = 
    excessSpyMonthlyAGV 
    |> List.averageBy (fun x -> x.Return)

// Test
mu2010s
|> should (equalWithin 0.005) 0.01

mu2010s

0.011143663482415371

> **Task:** If you are a mean-variance investor with a risk aversion parameter $\gamma=3$, what is the optimal weight of SPY in your portfolio over the period 2010-1 to 2019-12? Use your $\mu$ and $\sigma$ from the same period. Assign it to a value named `optimalWeight2010s`.
> 

Write your solution in the cell below.



In [18]:
// Solution here
let gamma = 3.0
let mu = mu2010s
let sigma =stdDev2010s
let optimalWeight2010s = mu / (gamma * sigma ** 2.0)
   
printfn $"The optimal weight of SPY in the portfolio over the period 2010-1 to 2019-12 is {optimalWeight2010s}."
optimalWeight2010s

// Test
if true then 
    optimalWeight2010s
    |> should (equalWithin 0.1) 2.75
    printfn "Test passed"


The optimal weight of SPY in the portfolio over the period 2010-1 to 2019-12 is 2,7774314778355396.
Test passed


> **Task:** Given that optimal weight in SPY for the 2010s, do you think the 2010s were a good decade to be invested in SPY? Why or why not? Explain using your estimate of the `optimalWeight2010s` as part of your justification.
> 

Write your solution in the cell below.



Based on the estimated optimal weight of 2.77 for SPY in the 2010s, it appears that SPY performed well during this decade. However, it's important to keep in mind that past performance does not guarantee future results. The optimal weight of 2.77 indicates that, on average, investing in SPY would have resulted in higher returns than investing in the risk-free asset. However, the optimal weight does not take into account other factors such as volatility and downside risk. It's possible that during the 2010s, there were periods of high volatility or significant market downturns that would have reduced the overall performance of SPY.

Therefore, while the optimal weight suggests that SPY was a good investment during the 2010s, it's important to consider a range of factors before making any investment decisions. Additionally, it's always a good idea to consult with a financial advisor before making any investment decisions.

> **Task:** The `optimalWeight2010s` is close to 2.75. Use a weight of 2.75 to invest in SPY excess returns from 2020-01 to the end of the sample (inclusive). What is the cumulative excess return of this portfolio? Assign it to a value named `cumulativeExcessReturn2020s`.
> 

Write your solution in the cell below.



In [77]:
// Solution here
// Filter the excess returns data to include only data from 2020-01 onwards
let excessReturnData2020s = excessSpy |> Seq.filter (fun d -> d.Date >= DateTime(2020, 1, 1))

// (1) Calculate the cumulative excess returns of the portfolio using a weight of 2.75 and (2) get final cumulative value
let cumulativeExcessReturn2020s =
    let mutable cumprod = 1.0
    let weight = 2.75
    let excessReturns = excessReturnData2020s |> Seq.map (fun row -> row.Return)
    for r in excessReturns do
        cumprod <- cumprod * (1.0 + weight * r)
    cumprod - 1.0
printfn $"The excess cumulative return of SPY from the beginning to the end of the sample period is {cumulativeExcessReturn2020s}."
cumulativeExcessReturn2020s

cumulativeExcessReturn2020s
|> should (equalWithin 0.05) 0.35
printfn "Test passed"


The excess cumulative return of SPY from the beginning to the end of the sample period is 0,33113974235227395.
Test passed


> **Task:** Plot the cumulative **excess** return of an investment in SPY levered to 2.75 from the 2020-01 to the end of the sample. The date should be the x-axis and the cumulative excess return should be the y-axis.
> 

Write your solution in the cell below.



In [78]:
// Solution here
// Extract the dates and cumulative excess returns as lists
let dates = excessReturnData2020s |> Seq.map (fun d -> d.Date) |> Seq.toList
let cumulativeExcessReturns = cumulativeExcessReturns2020s |> Seq.toList

// Merge dates and cumulative excess returns
let chartData = Seq.zip dates cumulativeExcessReturns

Chart.Line(chartData)

<!-- Plotly chart will be drawn inside this DIV -->

> **Task:** If you are a mean-variance investor with a risk aversion parameter $\gamma=3$, what is the optimal weight of SPY in your portfolio over the period 2020-1 to the end of the sample? Use $\mu$ and $\sigma$ estimated from 2020-01 to the end of the sample to form your estimate. Assign it to a value named `optimalWeight2020s`.
> 

Write your solution in the cell below.



In [79]:
// Solution here

let mu2020s = 
    excessReturnData2020s 
    |> Seq.averageBy (fun x -> x.Return)

let excessReturnData2020smonthly =
    excessReturnData2020s
    |> Seq.groupBy (fun x -> x.Date.Year, x.Date.Month)
    |> Seq.map (fun (_, group) -> group |> Seq.map (fun x -> x.Return) |> Seq.average)
    |> Seq.toList

let stdDev2020s =
    excessReturnData2020smonthly
    |> stDev 


let optimalWeight2020s = mu2020s / (gamma * stdDev2020s ** 2.0)
   
printfn $"The optimal weight of SPY in the portfolio from 2020-01 to the end of the sample is {optimalWeight2020s}."

//Test
if true then 
    optimalWeight2020s
    |> should (equalWithin 0.1) 0.7
    printfn "Test passed"

The optimal weight of SPY in the portfolio from 2020-01 to the end of the sample is 0,6998988379146324.
Test passed


> **Task:** Why is the optimal weight from the 2010s so different from the 2020s? Be specific and justify your answer using the data. What do we learn from this?
> 

Write your solution in the cell below.



The difference in the optimal weight for the risky asset between the 2010s (~ 2.77) and 2020s (~0.7) can be explained by changes in the characteristics of the asset's return over time. Specifically, the increase in the standard deviation of returns in the 2020s, coupled with a decrease in the average monthly returns, suggests that the risky asset became riskier and less attractive to investors.

In the 2010s, the risky asset had a lower standard deviation of monthly returns (~ 0.0366) and a higher average monthly return (~ 0.0111) compared to the period from 2020 onwards, where the standard deviation increased to approximately 0.0619 and the average monthly return decreased to approximately 0.008.

The calculation of the optimal weight takes into account the trade-off between expected return and risk, as measured by the standard deviation of returns. A higher standard deviation of returns increases the risk of the portfolio and therefore reduces the optimal weight that should be allocated to the risky asset. Conversely, a higher average monthly return increases the expected return of the portfolio and, therefore, increases the optimal weight that should be allocated to the risky asset. In this case, the increase in the standard deviation of returns in 2020 resulted in a lower optimal weight for the risky asset.

The difference in optimal weights between the two time periods highlights the importance of regularly reviewing investment decisions and adjusting portfolios based on changes in market conditions. Investors need to remain vigilant and respond to changes in the risk and return characteristics of their investments, to ensure that their portfolios remain aligned with their investment objectives and risk tolerance.